In [27]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import urllib3
import ssl
import requests.adapters
import urllib3.poolmanager

In [28]:
class CustomHttpAdapter(requests.adapters.HTTPAdapter):
    # "Transport adapter" that allows us to use custom ssl_context.

    def init(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().init(**kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize,
            block=block, ssl_context=self.ssl_context)


def get_legacy_session():
    ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
    ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE
    session = requests.session()
    session.mount('https://', CustomHttpAdapter(ctx))
    return session

In [29]:
failed_attempts=[]

In [31]:
urllib3.disable_warnings()

session = requests.Session()
headers_h = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
}

def link_getter(iter_obj):
    visited_links = set()  # To keep track of visited links
    with open('links.txt', 'a', encoding='utf-8') as file:
        for i in tqdm(range(1, iter_obj+1)):
            try:
                
                r = session.get(f'https://www.myhome.ge/ka/s/iyideba-bina-Tbilisshi/?deal_types=1&real_estate_types=1.2&cities=1&currency_id=1&CardView=1&page={i}', headers=headers_h, verify=False)
                soup = BeautifulSoup(r.content, 'html.parser')
                productlist = soup.find_all('div', class_='grid grid-cols-1 gap-5 md:grid-cols-2 lg:grid-cols-3 xl:grid-cols-4')
                
                for item in productlist:
                    for link in item.find_all('a', href=True):
                        link_href = link['href']
                        if link_href not in visited_links:  # Check if the link is not already processed
                            file.write(link_href + '\n')
                            visited_links.add(link_href)  # Add the link to the set of visited links
            except Exception as e:
                print(f"Failed to retrieve links from page {i}: {e}")
                failed_attempts.append(i)

link_getter(5)

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:02<00:00,  2.47it/s]


In [ ]:
print(visited_links)